In [4]:
import numpy as np
import pandas as pd
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

class BigramLM:
    def __init__(self):
        self.bigram_counts = defaultdict(lambda: defaultdict(int))
        self.unigram_counts = defaultdict(int)
        self.bigramDictionary = defaultdict(int)
        self.bigramProbabilities = defaultdict(int)
        self.vocab = set()
        self.discount = 0.75               # Kneser Ney Discount Factor
        self.probability_comparison_frame = pd.DataFrame()
        self.probability_comparison_frame_available = pd.DataFrame()

    def learn_model(self, dataset):
        for sentence in dataset:
            tokens = sentence
            tokens.append('$')  # Adding End of Sentence marker
            for i in range(len(tokens)):
                prev_token = tokens[i - 1]
                current_token = tokens[i]

                self.bigram_counts[prev_token][current_token] += 1
                self.unigram_counts[prev_token] += 1
                self.vocab.add(prev_token)
                self.bigramDictionary[(prev_token, current_token)] += 1

        self.calculate_probability()
        
    def calculate_probability(self):
        for bigram in self.bigramDictionary:
            self.bigramProbabilities[bigram] = (self.bigramDictionary[bigram]) / (self.unigram_counts[bigram[0]])
            
    def laplace_smoothing_probability(self, bigram):
        prefix_count = self.unigram_counts[bigram[0]]
        if bigram not in self.bigramDictionary:
            return ((1)/(prefix_count+len(self.vocab))) 
        
        bigram_count = self.bigramDictionary[bigram]
        return ((bigram_count+1)/(prefix_count+len(self.vocab)))      
        
    def kneserney_smoothing_probability(self, bigram):
        prefix_count = self.unigram_counts[bigram[0]]
        bigram_count = 0
        if bigram in self.bigramDictionary:
            bigram_count = self.bigramDictionary[bigram]
        
        bigram_types_with_suffix = len([x for x in self.bigramDictionary if x[1]==bigram[1]]) # fixed suffix, variable prefix(wi-1)
        bigram_types_with_prefix = len([x for x in self.bigramDictionary if x[0]==bigram[0]]) # fixed prefix, variable suffix(wi)
        total_bigram_types = len(self.bigramDictionary)
        
        discounted_prob = max(bigram_count - self.discount, 0) / prefix_count
        alpha_parameter = (self.discount / prefix_count) * bigram_types_with_prefix
        pcontinuation = bigram_types_with_suffix / total_bigram_types
        
        return discounted_prob + alpha_parameter * pcontinuation
            
    def generate_next_token(self, prev_token):
        if prev_token in self.bigram_counts:
            next_tokens = list(self.bigram_counts[prev_token].keys())
            probabilities = [self.bigramProbabilities[(prev_token, token)] for token in next_tokens]
            next_token = np.random.choice(next_tokens, p=probabilities)
            return next_token

        return None

    def generate_next_token_using_laplace(self, prev_token):
        next_tokens = [token for token in self.unigram_counts]
        
        probabilities = np.array([self.laplace_smoothing_probability((prev_token, token)) for token in next_tokens])
        
        next_token = np.random.choice(next_tokens, p=probabilities)
        return next_token

    def generate_next_token_using_kneserney(self, prev_token):
        next_tokens = [token for token in self.unigram_counts]
        
        probabilities = np.array([self.kneserney_smoothing_probability((prev_token, token)) for token in next_tokens])
        next_token = np.random.choice(next_tokens, p=probabilities)
        return next_token
        
    def generate_sentences_standard_bigram(self, num_samples, start_token="$"):
        if len(start_token.split()) ==0:
            start_token = "$"
            
        start_token = start_token.split()[-1]
        for x in range(num_samples):
            prev_token = start_token
            for i in range(10):
                next_token = self.generate_next_token(prev_token)
                print(prev_token, end=" ")
                prev_token = next_token
            print()
        print()
        
    def generate_sentences_laplace(self, num_samples, start_token="$"):
        if len(start_token.split()) ==0:
            start_token = "$"
            
        start_token = start_token.split()[-1]
        for x in range(num_samples):
            prev_token = start_token
            for i in range(10):
                next_token = self.generate_next_token_using_laplace(prev_token)
                print(prev_token, end=" ")
                prev_token = next_token
            print()
        print()
        
            
    def generate_sentences_kneserney(self, num_samples, start_token="$"):
        if len(start_token.split()) ==0:
            start_token = "$"
            
        start_token = start_token.split()[-1]
        for x in range(num_samples):
            prev_token = start_token
            for i in range(10):
                next_token = self.generate_next_token_using_kneserney(prev_token)
                print(prev_token, end=" ")
                prev_token = next_token
            print()
        print()
            
    def compare_probabilities(self, prev_token="$"):
        if len(prev_token.split()) ==0:
            prev_token = "$"
            
        prev_token = prev_token.split()[-1]
        if prev_token in self.bigram_counts:
            available_next_tokens = list(self.bigram_counts[prev_token].keys())
            probabilities_bigram = [self.bigramProbabilities[(prev_token, token)] for token in available_next_tokens]
            
            all_tokens = np.array([token for token in self.unigram_counts])
            probabilities_laplace = np.array([self.laplace_smoothing_probability((prev_token, token)) for token in self.unigram_counts])
            probabilities_kneserney = np.array([self.kneserney_smoothing_probability((prev_token, token)) for token in self.unigram_counts])
            
#             print("Standard Bigram Model Probabilities")
#             print(probabilities_bigram)

            # Set display options for float formatting
            pd.set_option('display.float_format', '{:.8f}'.format)
            # Create Probability DataFrame
            self.probability_comparison_frame = pd.DataFrame({
                'Token': all_tokens,
                'Laplace Probability': probabilities_laplace,
                'KneserNey Probability': probabilities_kneserney
            })
            
            self.probability_comparison_frame_available = self.probability_comparison_frame[self.probability_comparison_frame["Token"].isin(available_next_tokens)]
            self.probability_comparison_frame_available["Bigram Probability"] = probabilities_bigram
            print("Probability Comparison of All Tokens")
            print(self.probability_comparison_frame)
            
            print("\nProbability Comparison of Next Available Tokens in Bigram Dictionary")
            print(self.probability_comparison_frame_available)
            
#             print("\nLaplace Bigram Model Probabilities")
#             print(list(probabilities_laplace))
            
#             print("\nKneserNey Bigram Model Probabilities")
#             print(list(probabilities_kneserney))
            return

        return None

with open("corpus.txt", "r", encoding="utf-8") as file:
    corpus = [line.strip().split() for line in file] 

bigram_model = BigramLM()
bigram_model.learn_model(corpus)

# bigram_model.generate_sentences_standard_bigram("of", 10)

# bigram_model.generate_sentences_laplace(10, " ")

# bigram_model.generate_sentences_kneserney("of", 10)

bigram_model.compare_probabilities()


Probability Comparison of All Tokens
           Token  Laplace Probability  KneserNey Probability
0              $           0.00012771             0.00023928
1              i           0.26934866             0.87810280
2          stand           0.00012771             0.00000156
3           here           0.00012771             0.00000623
4           feel           0.00012771             0.00002181
...          ...                  ...                    ...
5425      google           0.00012771             0.00000019
5426  stellarium           0.00012771             0.00000019
5427       theyd           0.00012771             0.00000019
5428       peter           0.00012771             0.00000019
5429      robbed           0.00012771             0.00000019

[5430 rows x 3 columns]

Probability Comparison of Next Available Tokens in Bigram Dictionary
        Token  Laplace Probability  KneserNey Probability  Bigram Probability
1           i           0.26934866             0.87810280 

## Argument:

As we can see, their is a huge difference in Laplace probabilities and original Bigram probabilities. This is because Laplace smoothing steals a large amount of probabilities from non-zero counts of tokens to distribute it into tokens with zero occurrences. Thus in Laplace smoothing, the reconstructed counts of non-zero tokens could change largely sometimes by a factor of 10 from their original counts. For example, token "i" has a Laplace probability of 0.269 compared to its original probability of 0.87 i.e. a 1/3rd of difference from original probability while KneserNey has somewhat managed to maintain the probability in proportion with original probability.
Thus Laplace haven't worked in our Bigram model well and doesn't work well in general for n-grams.

On the other hand, Kneser-Ney has somewhat maintained the probabilities with original Bigram probabilities compared to Laplace.

Thus KneserNey smoothing is better than Laplace in this case